In [1]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

!git clone https://github.com/SKT-AI/KoGPT2.git

%cd ./KoGPT2/
!ls

!pip install -r requirements.txt
!pip install .

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [27.5 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [817 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubunt

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os
import csv
from tqdm import tqdm
import re

In [0]:
def choose_from_top(probs, n=5):
  ind = np.argpartition(probs, -n)[-n:]
  top_prob = probs[ind]
  top_prob = top_prob / np.sum(top_prob)
  choice = np.random.choice(n, 1, p = top_prob)
  token_id = ind[choice][0]
  return int(token_id)

In [0]:
class NovelDataset(Dataset):
  def __init__(self, novel_dataset_path="/content/drive/My Drive/4IR/data"):
    super().__init__()

    novel_path = os.path.join(novel_dataset_path, "nv_ft_split_del_col.csv")
    self.novel_list = []
    self.end_of_text_token = "<|endoftext|>"

    df = pd.read_csv(novel_path, header=None)


    for row in range(len(df)):
      novel_str = df.loc[row, 1]+self.end_of_text_token
      self.novel_list.append(novel_str)
  
  def __len__(self):
    return len(self.novel_list)

  def __getitem__(self, item):
    return self.novel_list[item]

In [0]:
def generate_some_text(input_str, text_len = 200):
  curs_id = torch.tensor(tokenizer.encode(input_str, max_length=1000)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in tqdm(range(text_len)):
      outputs = model(curs_id, labels=curs_id)
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to("cpu").numpy(), n=10)
      curs_id = torch.cat([curs_id, torch.ones((1,1)).long().to(device)*next_token_id], dim = 1)
    
    output_list = list(curs_id.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    print(output_text)

In [0]:
def return_latest_epoch(models_folder):
  #models_folder = "/content/drive/My Drive/4IR/trained_model"
  epoch_list = [int(re.findall("\d", re.findall("_\d.", model_name)[0])[0]) for model_name in os.listdir(models_folder)]
  epoch_list.sort()
  return epoch_list[-1]

In [7]:
device = 'cpu'
if torch.cuda.is_available():
  print("You can use GPU~!")
  device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model = model.to(device)

You can use GPU~!


In [0]:
dataset = NovelDataset()
novel_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [0]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400

device = 'cpu'
if torch.cuda.is_available():
  device = "cuda"

In [0]:
model = model.to(device)

#"/content/drive/My Drive/4IR/trained_model"
#models_folder = "trained_models"
models_folder = "/content/drive/My Drive/4IR/trained_model"

#MODEL_EPOCH = 4
MODEL_EPOCH = return_latest_epoch(models_folder)

model_path = os.path.join(models_folder, f"gpt2_novel_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

model.train()

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_novel_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
  os.mkdir(models_folder)

for epoch in range(MODEL_EPOCH+1, MODEL_EPOCH+EPOCHS+1):
  print(f"EPOCH {epoch} started"+"="*30)
  for idx, novel in tqdm(enumerate(novel_loader)):
    novel_tens = torch.tensor(tokenizer.encode(novel[0], max_length=6000)).unsqueeze(0).to(device)

    if novel_tens.size()[1] > MAX_SEQ_LEN:
      continue

    if not torch.is_tensor(tmp_novel_tens):
      tmp_novel_tens = novel_tens
      continue
    else:
      if tmp_novel_tens.size()[1] + novel_tens.size()[1] > MAX_SEQ_LEN:
        work_novel_tens = tmp_novel_tens
        tmp_novel_tens = novel_tens
      else:
        tmp_novel_tens = torch.cat([tmp_novel_tens, novel_tens[:,1:]], dim=1)
        continue
    
    outputs = model(work_novel_tens, labels=work_novel_tens)
    loss, logits = outputs[:2]
    loss.backward()
    sum_loss = sum_loss + loss.detach().data

    proc_seq_count = proc_seq_count + 1
    if proc_seq_count == BATCH_SIZE:
      proc_seq_count = 0
      batch_count += 1
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      model.zero_grad()

    if batch_count == 100:
      print(f"sum loss {sum_loss}")
      batch_count = 0
      sum_loss = 0.0
  torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_novel_{epoch}.pt"))


0it [00:00, ?it/s]

EPOCH 5 started==============================


5097it [03:15, 25.64it/s]

sum loss 3180.04052734375


8773it [05:41, 29.62it/s]

In [0]:
torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_novel_{epoch}.pt"))

NameError: ignored

In [0]:

models_folder = "/content/drive/My Drive/4IR/trained_model"

with open(os.path.join(models_folder,"test.txt"), "w") as file:
  file.write("hello")
